## From last time

- we had our simple bitcask like append only database
- it consisted of an in-memory dictionary and a append-only file
- the append only file is an example of a write-ahead log (WAL) as well!

Now for our **LSM-tree**...this is the architecture used in **LevelDB**, etc:

1. when a write comes in add it first to the WAL.
2. to an in-memory balanced tree structure, called a memtable, for example a red-black tree. The WAL above's sole purpose is to reconstruct this tree in the face of a crash
3. At some threshold we write the tree into a SSTable (sorted string table) file. This becomes the most recent segment of the database, and the memtable is cleared out for reuse
4. Searches go from memtable to these segmants, one by one, with a bloom filter being used to tell if the key was never there. 
5. we run merging and compaction in the background in the spirit of a mergesort and to discard overwritten or deleted values.

(what about the dictionary in memory with offsets: that depends on how big a dictionary you want to keep. You dont need it because of the memtable, but it might be worth making several sparse indexes, one for each segment. while it wont tell you if a key is in that segment, it will tell you where to seek to and do a linear scan from)

## Transaction Processing or Analytics?

- Also known as OLTP vs OLAP/Warehousing
- small query size vs aggregates over large ones
- random writes from user input vs ordered ETL/stream
- end user (amazon site) vs analyst (you)
- GB to TB vs TB to PB

![](https://dl.dropboxusercontent.com/u/75194/ETL.png)

(from designing data intensive applications)

### Handling the different workloads

- for smaller sizes any relational db will do
- currently vendors focus on one or the other, not both
- MS and SAP HANA support both but with different storage engines
- OLTP need to be highly available, low latency
- SQL (or any Pandasish syntax) is good for drilling down
- warehousing: star schema with very wide fact table, but typically you focus on few columns at a time
- btree indexes for oltp, bitmaps + btree for warehouse

### Column oriented storage

![](https://www.simple-talk.com/iwritefor/articlefiles/1844-f4cc85b0-9ddb-44cc-93ef-a742fcc4f279.jpeg)

- store values from each column together in separate storage
- lends itself to compression with bitmap indexes and run-length encoding
- this involves choosing an appropriate sort order
- the index then can be the data (great for IN and AND queries): there is no pointers to "elsewhere"
- compressed indexes can fit into cache and are usable by iterators
- bitwise AND/OR can be done with vector processing
- several different sort orders can be redundantly stored
- writing is harder: updating a row touches many column files
- but you can write an in-memory front sorted store (row or column), and eventually merge onto the disk (Binary Tree!) (Vertica does this)

![](https://www.simple-talk.com/iwritefor/articlefiles/1844-4e2482bb-aaff-4ebd-8900-1946560479af.jpeg)

(images from https://www.simple-talk.com/sql/database-administration/columnstore-indexes-in-sql-server-2012/)



### Data cubes

- Basically a histogram of counts in bins for multiple fields
- can give you fast marginals and conditionals in any combination of dimensions
- expensive to update so only used for warehousing
- such histograms are used by query optimizers as well

## Back to indexes: B and B+-trees

The log structured indexes wrote things sequentially, into segments, typically multiple MB sizes

![](https://dl.dropboxusercontent.com/u/75194/btree1q.png)

(from https://loveforprogramming.quora.com/Memory-locality-the-magic-of-B-Trees)

- "A linked sorted distributed range array with predefined sub array size which allows searches, sequential access, insertions, and deletions in logarithmic time. "
- it is a generalization of a binary tree
- but the branching factor is much higher, and the depth thus smaller
- brees break database into pages, and read-or-write one page at a time. A page is about 4k in size
- leaf pages contain all the values and may represent a clustered index
- the pointers in a btree are disk based pointers

![](https://dl.dropboxusercontent.com/u/75194/btree1.png)
(from designing data intensive applications)

When we update a key, a split can happen

![](https://dl.dropboxusercontent.com/u/75194/btree2.png)

(from designing data intensive applications)

This is an in-place modificaltion unlike what we had earlier. The data structure is mutable. This can cause issues for transactions, and must be dealt with. One can create immutable b-trees, and lmdb, the database inside of openldap, does this. It uses a copy-on-write schee which writes new pages elsewhere

Both splits and writing in-place are dangerous, so its normal for b-tree implementations to have a WAL, or write ahead log (such a log can also be used to manage transactions). Every operation on the btree is appended to this log file.

In B+ trees, pointers amonst the leaf nodes make for an easier linear scan.

![](https://dl.dropboxusercontent.com/u/75194/btree2q.png)

(from https://loveforprogramming.quora.com/Memory-locality-the-magic-of-B-Trees)

### B-tree vs LSM tree

- critical difference: btrees update in-place!
- comparable on random reads
- compaction can affect performancs in LSM trees
- LSM tree good on random writes as it makes the writes sequential
- B-tree good for transactions; at most one place where things are: you just need to lock a section of the tree. Still, concurrency can be complex.

### A bit more detail

- what is a WAL based B-tree

splits are dangerous. updating in place is dangerous. Thus many btree situations use a WAL. concurrency is complex as things must be changed in place

![](https://dl.dropboxusercontent.com/u/75194/wal1.png)
![](https://dl.dropboxusercontent.com/u/75194/wal2.png)
![](https://dl.dropboxusercontent.com/u/75194/wal3.png)


- what is Append-Only

![](https://dl.dropboxusercontent.com/u/75194/append1.png)

- What is copy on write

![](https://dl.dropboxusercontent.com/u/75194/cow1.png)
![](https://dl.dropboxusercontent.com/u/75194/cow2.png)
![](https://dl.dropboxusercontent.com/u/75194/cow3.png)

See http://symas.com/mdb/20141120-BuildStuff-Lightning.pdf

## What is an index and what is a database?

- key-value indexes suppose unique keys and are thus like primary keys in a relational model
- you can also have secondary keys or multi-column keys which can be created by some kind of concatenation or a multi-dimensional r-tree, 
- in a k-v database, the value is stored in the index and we are done. Usually in rdbms, the rows are stored in a heap file to avoid duplication, and the value of the key is **a pointer to a location in the heapfile**. This recalls our bitcask like data structure.
- in mysql's innodb engine, the pk is a **clustered index**, while the secondary keys point to the pk.
- a **covering index** stores certain columns in the index. Of-course in a columnar situation, the column is the index when we choose that columns ordering.

## Immutable BST

We'll study the binary search tree as a proxy for the btree. Its not the best choice, as rebalancing and moving keys around creates havoc with seeking on disk, rather than being localizes to some pages, but its much more understandable.

![](https://dl.dropboxusercontent.com/u/75194/immutablebst.png)

(from purely functional data structures by Okasaki: great book BTW, with examples of balanced trees as well)

## LAB: An append-only, disk based, BST

Simplified and taken from http://aosabook.org/en/500L/dbdb-dog-bed-database.html, which you should read. The whole book is good, Guido's co-routineexample was from there too, and there is an example of a Transactonal Datomic-like database written in clojure there, as well.

In real life this should be a balanced tree, and you could do that as a project. This might help:

http://scottlobdell.me/2016/02/purely-functional-red-black-trees-python/

**Your job is to fill in the "your code here" parts**. 

Our assumption, again, is that keys and values are strings.

We start by creating a "disk reference", henceforth knows as a "ref" to a object. Currently we care aboutthe actual value of a key.

In [20]:
class ValueRef(object):
    " a reference to a string value on disk"
    def __init__(self, referent=None, address=0):
        self._referent = referent #value to store
        self._address = address #address to store at
        
    @property
    def address(self):
        return self._address
    
    def prepare_to_store(self, storage):
        pass

    @staticmethod
    def referent_to_bytes(referent):
        return referent.encode('utf-8')

    @staticmethod
    def bytes_to_referent(bytes):
        return bytes.decode('utf-8')

    
    def get(self, storage):
        "read bytes for value from disk"
        if self._referent is None and self._address:
            self._referent = self.bytes_to_referent(storage.read(self._address))
        return self._referent

    def store(self, storage):
        "store bytes for value to disk"
        #called by BinaryNode.store_refs
        if self._referent is not None and not self._address:
            self.prepare_to_store(storage)
            self._address = storage.write(self.referent_to_bytes(self._referent))




We inherit from the `ValueRef` to create a `BinaryNodeRef` that actually stores the nodes. Note that we use pickle as out serializations cheme: this is not particularly fast but will do the job. You could implement a far tighter serialization scheme like we did last time.

In [21]:
import pickle
class BinaryNodeRef(ValueRef):
    "reference to a btree node on disk"
    
    #calls the BinaryNode's store_refs
    def prepare_to_store(self, storage):
        "have a node store its refs"
        if self._referent:
            self._referent.store_refs(storage)

    @staticmethod
    def referent_to_bytes(referent):
        "use pickle to convert node to bytes"
        return pickle.dumps({
            'left': referent.left_ref.address,
            'key': referent.key,
            'value': referent.value_ref.address,
            'right': referent.right_ref.address,
        })

    @staticmethod
    def bytes_to_referent(string):
        "unpickle bytes to get a node object"
        d = pickle.loads(string)
        return BinaryNode(
            BinaryNodeRef(address=d['left']),
            d['key'],
            ValueRef(address=d['value']),
            BinaryNodeRef(address=d['right']),
        )
    


Next we create a node. Notice how it looks just like a binary tree node, but instead of pointers to left and right nodes, it keeps noderefs and valuerefs. This extra indirection enables us to store really large binary trees at the expense of speed. Indeed we could get the best of all worlds by memorymapping the database file.

In [22]:
class BinaryNode(object):
    @classmethod
    def from_node(cls, node, **kwargs):
        "clone a node with some changes from another one"
        return cls(
            left_ref=kwargs.get('left_ref', node.left_ref),
            key=kwargs.get('key', node.key),
            value_ref=kwargs.get('value_ref', node.value_ref),
            right_ref=kwargs.get('right_ref', node.right_ref),
        )

    def __init__(self, left_ref, key, value_ref, right_ref):
        self.left_ref = left_ref
        self.key = key
        self.value_ref = value_ref
        self.right_ref = right_ref

    def store_refs(self, storage):
        "method for a node to store all of its stuff"
        self.value_ref.store(storage)
        #calls BinaryNodeRef.store. which calls
        #BinaryNodeRef.prepate_to_store
        #which calls this again and recursively stores
        #the whole tree
        self.left_ref.store(storage)
        self.right_ref.store(storage)

Finally the tree is made of nodes with their refs to nodes and so on and so forth. You need to implement `get`.

In [23]:
class BinaryTree(object):
    "Immutable Binary Tree class. Constructs new tree on changes"
    def __init__(self, storage):
        self._storage = storage
        self._refresh_tree_ref()

    def commit(self):
        "changes are final only when committed"
        #triggers BinaryNodeRef.store
        self._tree_ref.store(self._storage)
        #make sure address of new tree is stored
        self._storage.commit_root_address(self._tree_ref.address)

    def _refresh_tree_ref(self):
        "get reference to new tree if it has changed"
        self._tree_ref = BinaryNodeRef(
            address=self._storage.get_root_address())

    def get(self, key):
        "get value for a key"
        #if tree is not locked by another writer
        #refresh the references and get new tree if needed
        if not self._storage.locked:
            self._refresh_tree_ref()
        #your code here
        #get the top level node
        #traverse until you find appropriate node
        #if key is not found raise KeyError

    def set(self, key, value):
        "set a new value in the tree. will cause a new tree"
        #try to lock the tree. If we succeed make sure
        #we dont lose updates from any other process
        if self._storage.lock():
            self._refresh_tree_ref()
        #get current top-level node and make a value-ref
        node = self._follow(self._tree_ref)
        value_ref = ValueRef(value)
        #insert and get new tree ref
        self._tree_ref = self._insert(node, key, value_ref)
        
    
    def _insert(self, node, key, value_ref):
        "insert a new node creating a new path from root"
        #create a tree ifnthere was none so far
        if node is None:
            new_node = BinaryNode(
                BinaryNodeRef(), key, value_ref, BinaryNodeRef())
        elif key < node.key:
            new_node = BinaryNode.from_node(
                node,
                left_ref=self._insert(
                    self._follow(node.left_ref), key, value_ref))
        elif key > node.key:
            new_node = BinaryNode.from_node(
                node,
                right_ref=self._insert(
                    self._follow(node.right_ref), key, value_ref))
        else: #create a new node to represent this data
            new_node = BinaryNode.from_node(node, value_ref=value_ref)
        return BinaryNodeRef(referent=new_node)

    def delete(self, key):
        "delete node with key, creating new tree and path"
        if self._storage.lock():
            self._refresh_tree_ref()
        node = self._follow(self._tree_ref)
        self._tree_ref = self._delete(node, key)
        
    def _delete(self, node, key):
        "underlying delete implementation"
        if node is None:
            raise KeyError
        elif key < node.key:
            new_node = BinaryNode.from_node(
                node,
                left_ref=self._delete(
                    self._follow(node.left_ref), key))
        elif key > node.key:
            new_node = BinaryNode.from_node(
                node,
                right_ref=self._delete(
                    self._follow(node.right_ref), key))
        else:
            left = self._follow(node.left_ref)
            right = self._follow(node.right_ref)
            if left and right:
                replacement = self._find_max(left)
                left_ref = self._delete(
                    self._follow(node.left_ref), replacement.key)
                new_node = BinaryNode(
                    left_ref,
                    replacement.key,
                    replacement.value_ref,
                    node.right_ref,
                )
            elif left:
                return node.left_ref
            else:
                return node.right_ref
        return BinaryNodeRef(referent=new_node)

    def _follow(self, ref):
        "get a node from a reference"
        #calls BinaryNodeRef.get
        return ref.get(self._storage)
    
    def _find_max(self, node):
        while True:
            next_node = self._follow(node.right_ref)
            if next_node is None:
                return node
            node = next_node

We implement storage in a file. unlike in btrees where we might lock a key range, we lock the whole file on writes. In a more sophisticated software implementation of locks, we could lock subtrees instead.

You need to implement `write` in here.

In [24]:
import os
import struct

import portalocker


class Storage(object):
    SUPERBLOCK_SIZE = 4096
    INTEGER_FORMAT = "!Q"
    INTEGER_LENGTH = 8

    def __init__(self, f):
        self._f = f
        self.locked = False
        #we ensure that we start in a sector boundary
        self._ensure_superblock()

    def _ensure_superblock(self):
        "guarantee that the next write will start on a sector boundary"
        self.lock()
        self._seek_end()
        end_address = self._f.tell()
        if end_address < self.SUPERBLOCK_SIZE:
            self._f.write(b'\x00' * (self.SUPERBLOCK_SIZE - end_address))
        self.unlock()

    def lock(self):
        "if not locked, lock the file for writing"
        if not self.locked:
            portalocker.lock(self._f, portalocker.LOCK_EX)
            self.locked = True
            return True
        else:
            return False

    def unlock(self):
        if self.locked:
            self._f.flush()
            portalocker.unlock(self._f)
            self.locked = False

    def _seek_end(self):
        self._f.seek(0, os.SEEK_END)

    def _seek_superblock(self):
        "go to beginning of file which is on sec boundary"
        self._f.seek(0)

    def _bytes_to_integer(self, integer_bytes):
        return struct.unpack(self.INTEGER_FORMAT, integer_bytes)[0]

    def _integer_to_bytes(self, integer):
        return struct.pack(self.INTEGER_FORMAT, integer)

    def _read_integer(self):
        return self._bytes_to_integer(self._f.read(self.INTEGER_LENGTH))

    def _write_integer(self, integer):
        self.lock()
        self._f.write(self._integer_to_bytes(integer))

    def write(self, data):
        "write data to disk, returning the adress at which you wrote it"
        #first lock, get to end, get address to return, write size
        #write data, unlock
        #your code here
        

    def read(self, address):
        self._f.seek(address)
        length = self._read_integer()
        data = self._f.read(length)
        return data

    def commit_root_address(self, root_address):
        self.lock()
        self._f.flush()
        #make sure you write root address at position 0
        self._seek_superblock()
        #write is atomic because we store the address on a sector boundary.
        self._write_integer(root_address)
        self._f.flush()
        self.unlock()

    def get_root_address(self):
        #read the first integer in the file
        #your code here
        self._seek_superblock()
        root_address = self._read_integer()
        return root_address

    def close(self):
        self.unlock()
        self._f.close()

    @property
    def closed(self):
        return self._f.closed

The database is just a thin layer over the tree

In [25]:
class DBDB(object):

    def __init__(self, f):
        self._storage = Storage(f)
        self._tree = BinaryTree(self._storage)

    def _assert_not_closed(self):
        if self._storage.closed:
            raise ValueError('Database closed.')

    def close(self):
        self._storage.close()

    def commit(self):
        self._assert_not_closed()
        self._tree.commit()

    def get(self, key):
        self._assert_not_closed()
        return self._tree.get(key)

    def set(self, key, value):
        self._assert_not_closed()
        return self._tree.set(key, value)

    def delete(self, key):
        self._assert_not_closed()
        return self._tree.delete(key)

And here is the `connect`:

In [26]:
def connect(dbname):
    try:
        f = open(dbname, 'r+b')
    except IOError:
        fd = os.open(dbname, os.O_RDWR | os.O_CREAT)
        f = os.fdopen(fd, 'r+b')
    return DBDB(f)

Lets play. The following ops should be reasonably clear from last time!

In [27]:
!rm /tmp/test2.dbdb

In [28]:

db = connect("/tmp/test2.dbdb")

In [29]:
db.set("rahul", "aged")
db.set("pavlos", "aged")
db.set("kobe", "stillyoung")

In [30]:
db.close()

In [31]:
db = connect("/tmp/test2.dbdb")
db.get("rahul")

KeyError: 

In [32]:
db.set("rahul", "aged")
db.set("pavlos", "aged")
db.set("kobe", "stillyoung")
db.commit()

In [33]:
db.close()
db = connect("/tmp/test2.dbdb")
db.get("rahul")

'aged'

In [34]:
db.set("rahul", "young")
db.get("rahul")

'young'

In [35]:
db.close()
db = connect("/tmp/test2.dbdb")
db.get("rahul")

'aged'

In [36]:
db.set("rahul", "young")
db.commit()
db.close()
db = connect("/tmp/test2.dbdb")
db.get("rahul")

'young'

In [37]:
db.delete("pavlos")
db.commit()